In [19]:
import os
import pandas as pd
import numpy as np
from datetime import datetime as datatime
from pathlib import Path
import torch
import torch.nn as nn
from PIL import Image
# import torch.optim as optim
from torchvision import transforms
from torchvision import models
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix,classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler

from tqdm import tqdm


In [20]:
# import zipfile
# import os

# # 要解压的 zip 文件路径
# train_zip = ('./COMP90086_2024_Project_train.zip')
# test_zip = ('./COMP90086_2024_Project_test.zip')

# # 解压后的目标文件夹（如果不存在则创建）
# train_folder = 'COMP90086_2024_Project_train'
# os.makedirs(train_folder, exist_ok=True)
# test_folder = 'COMP90086_2024_Project_test'
# os.makedirs(test_folder, exist_ok=True)

# # 打开并解压 zip 文件

# with zipfile.ZipFile(train_zip, 'r') as zip_ref:
#     zip_ref.extractall(train_folder)

# with zipfile.ZipFile(test_zip, 'r') as zip_ref:
#     zip_ref.extractall(test_folder)
    
# print(f"Files extracted to {train_folder}")
# print(f"Files extracted to {test_folder}")

Files extracted to COMP90086_2024_Project_train
Files extracted to COMP90086_2024_Project_test


In [21]:
csv_path = './COMP90086_2024_Project_train/train.csv'
img_dir = './COMP90086_2024_Project_train/train'

## The data set file path is defined here.

In [22]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

'''
In this function block, the funciton is defined to do
1. def__init__() : get the image from the trainning data path
2. def __len__() : get the size of hte dataset
3. def __getitem__() : get the ralated image the correct dataset
'''
class BlockStackDataset(Dataset):
  '''
  Return the image and its label. If not access or not exist, raise error.
  '''
  def __init__(self, data_frame, img_dir, transform = None):
    self.data_frame = data_frame
    self.img_dir = img_dir
    self.transform = transform

    if not os.path.exists(self.img_dir):
      raise ValueError(f"File path {self.img_dir} not exisits!")
    if not os.access(self.img_dir, os.R_OK):
      raise ValueError(f"File path {self.img_dir} is not readable!")
  '''
  Return the size of the dataset
  '''
  def __len__(self):
    return len(self.data_frame)

  '''
  Read the image and according to its id, get the related data in the train.csv.

  Then return the corresponding image , stable_height, instability_type dataset provided.
  '''
  def __getitem__(self, idx):
    img_name = os.path.join(self.img_dir, str(self.data_frame.iloc[idx, 0])) # the first column in the train == image name
    image = Image.open(img_name + ".jpg")
    label = self.data_frame.iloc[idx,-1] # stable_height/instability_type
    # print("image id: ", self.data_frame.iloc[idx,0])
    # print("Whole line: ", self.data_frame.iloc[idx,:].tolist())

    if self.transform:
      image = self.transform(image)

    return image, label



In [23]:
class FineTunedGoogLeNet(nn.Module):
    def __init__(self):
        super(FineTunedGoogLeNet, self).__init__()
        # load the pre-trained model: gogglenet
        self.googlenet = models.googlenet(weights = models.GoogLeNet_Weights.IMAGENET1K_V1)

        num_ftrs = self.googlenet.fc.in_features
        self.googlenet.fc = nn.Identity()

        self.fc = nn.Sequential(        
            nn.Linear(num_ftrs, 256),
            nn.BatchNorm1d(256),  # 添加 Batch Normalization
           # nn.LeakyReLU(negative_slope=0.01),
            nn.ReLU(),
            nn.Dropout(0.25),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),  # 添加 Batch Normalization
        #    nn.LeakyReLU(negative_slope=0.01),
            nn.ReLU(),
            nn.Dropout(0.25),
            
            nn.Linear(128, 6)

        )
    def forward(self, x):
      x = self.googlenet(x)
      x = self.fc(x)
      return x

In [24]:
df = pd.read_csv('./COMP90086_2024_Project_train/train.csv')
df.head()

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height
0,54,2,1,3,1,1,2
1,173,1,1,4,1,2,1
2,245,1,1,4,1,2,1
3,465,2,1,5,0,1,5
4,611,2,1,3,1,1,1


## Trainner

In [25]:
'''
The function here helps to create the logs recording the experiment results.
'''
def create_log_dir():
  current_time = datatime.now().strftime("%Y%m%d-%H%M%S")
  # log_dir = f'/content/drive/MyDrive/CV final project/runs/experiment_{current_time}'
  # solution_dir = f'/content/drive/MyDrive/CV final project/trained_models/experiment_{current_time}'

  log_dir = f'./runs/experiment_{current_time}'
  solution_dir = f'./runs/experiment_{current_time}'
  if not os.path.exists(log_dir):
    os.makedirs(log_dir)

  if not os.path.exists(solution_dir):
    os.makedirs(solution_dir)

  return log_dir, solution_dir


In [26]:

# Define models: googlenet

# class TunnedBlockStackNet8(nn.Module):
#     def __init__(self):
#         super(TunnedBlockStackNet8, self).__init__()
#         # load the pre-trained model: gogglenet
#         self.googlenet = models.googlenet(weights = models.GoogLeNet_Weights.IMAGENET1K_V1)

#         num_ftrs = self.googlenet.fc.in_features
#         self.googlenet.fc = nn.Identity()

#         self.fc = nn.Sequential(
#             nn.Linear(num_ftrs, 256),
#             nn.ReLU(),
#             nn.Dropout(0.25),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Dropout(0.25),
#             nn.Linear(128, 6)

#         )
#     def forward(self, x):
#       x = self.googlenet(x)
#       x = self.fc(x)
#       return x

In [27]:
class ClassificationBlockStackTrainer1:
  '''
  The function here helps to initalize the parameters used in the models and pre-process the image
  '''
  def __init__(self, csv_path, img_dir, model, stratify_column='stable_height', test_size=0.2,
                 batch_size=32, num_epochs=10, learning_rate=0.001,random_state=42):
        self.img_dir = img_dir
        self.stratify_column = stratify_column
        self.test_size = test_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.model = model

        # load the train dataset
        self.data_frame = pd.read_csv(csv_path)

        # split data into train and validation dataset
        self.train_data, self.val_data, self.train_ids, self.valid_ids = self.split_dataset()
        self.dynamic_train = self.train_data.copy()

       # self.train_data = self.dynamic_oversamp(self.train_data)

        # pre-processing the images
        self.transform = transforms.Compose([
            # transforms.Resize((512, 512)),

              #ADD MORE TRANSFORM METHODS HERE

            transforms.RandomHorizontalFlip(),## ADDDED
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)), ## ADDDED
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),## ADDDED
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


        self.train_loader = self.create_dataloader(self.train_data, self.transform)
        self.val_loader = self.create_dataloader(self.val_data, self.transform, shuffle=False)


        # use the gpu
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # set the loss weight
        '''
        Changed some thing here : to see the results
        1. add the class_weights to each class labels with different weights,
           more lables occurece with lower weight, few label occurence with higher weight;
           without the weights, all the labels will be treated equally.

        2. used to apply L2 regularization (also called weight decay).
        The primary purpose of weight decay is to prevent overfitting by penalizing large weights.
         It adds a penalty to the loss function based on the size of the weights
         helps it generalize better to unseen data.
         Lnew = Lold + weight_decay * sum(weight^2)

        3. add scheduler to Reduces the learning rate after every step_size epochs.
          After every 4 epochs, the learning rate will be multiplied by gamma (0.1 here), reducing it by 90%.
        '''
        class_weights = torch.tensor([100/25 , 100/25, 100/20,  100/15, 100/10,  100/5], device= self.device)
        self.criterion = nn.CrossEntropyLoss(weight=class_weights) # CrossEntropy for multi categorical-label predication
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate , weight_decay=0.0001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size = 15, gamma=0.1)


  def dynamic_oversamp(self, train_df, epoch):
    if epoch % 3 == 0:
        over_random = 333
    elif epoch % 3 == 1:
        over_random = 111
    else:
        over_random = 222
        
    hard_df = self.train_data[(train_df['type']==2) &
                              (train_df['instability_type']==0)]

    oversample_hard = resample(hard_df,replace=True,n_samples=500, random_state = over_random)

    hard_df2 = self.train_data[(train_df['type']==2) &
                              (train_df['instability_type']==1)]
    oversample_hard2 = resample(hard_df2,replace=True,n_samples=500, random_state = over_random)

    hard_df3 = self.train_data[(train_df['type']==2) &
                              (train_df['instability_type']==2)]

    oversample_hard3 = resample(hard_df3,replace=True,n_samples=200, random_state = over_random)
    
    conca_df = pd.concat([train_df,oversample_hard,oversample_hard2, oversample_hard3])
   # self.train_data = conca_df
    # 打乱顺序
    conca_df = conca_df.sample(frac=1)
    print(conca_df['stable_height'].value_counts())
    return conca_df
        
  '''
  The function helps to split the data set into the training and validation dataset according to the
  size pre-determined.
  '''

  def split_dataset(self):
    split = StratifiedShuffleSplit(n_splits=1, test_size=self.test_size, random_state=42)
    train_ids = []
    valid_ids = []
    for train_idx, val_idx in split.split(self.data_frame, self.data_frame[self.stratify_column]):
      train_data = self.data_frame.iloc[train_idx]
      val_data = self.data_frame.iloc[val_idx]
      train_ids.append(train_idx)
      valid_ids.append(val_idx)
    print(f"Train dataset size: {len(train_data)}",
       f"Validation dataset size: {len(val_data)}",
       f"length of train_ids{(len(train_ids))}",
       f"length of valid_ids{(len(valid_ids))}")
    return train_data, val_data, train_ids, valid_ids


  '''
  The function helps to loda the image
  '''
  def create_dataloader(self, data_frame, transform, shuffle=True):
    dataset = BlockStackDataset(data_frame, self.img_dir, transform=transform) # transform 可以用来数据增强
    return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)


  def generate_classification_report(self, outputs, labels):
    predicted = outputs # 'outputs' is already a numpy array after prediction
    labels = labels
    print(classification_report(labels, predicted, zero_division=0))



  def calculate_confusion_matrix(self, outputs, labels):
    predicted = outputs
    labels = labels
    matrix = confusion_matrix(labels, predicted)
    print(matrix)


  def validate(self):
    self.model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_labels = []  # add all the lables
    all_predictions = []  # add all the prediction

    with torch.no_grad():
        for inputs, labels in self.val_loader:
            inputs, labels = inputs.to(self.device), labels.to(self.device).long()
            labels = labels - 1
            outputs = self.model(inputs)
            predicted = torch.argmax(outputs, 1)

            # collecting all the lables and predictions
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            # calculate the loss
            loss = self.criterion(outputs, labels)
            val_loss += loss.item()

            # calculate the correct predication
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    # calculate the accuracy rate
    val_accuracy = correct_predictions / total_samples
    self.generate_classification_report(np.array(all_predictions), np.array(all_labels))
    self.calculate_confusion_matrix(np.array(all_predictions), np.array(all_labels))
    return val_loss/len(self.val_loader),val_accuracy, all_labels, all_predictions


  '''
  The function here is used as the main training function on the image by using the pre-definned models in
  hte first model class.
  '''

  def train(self):
      _, solution_dir = create_log_dir()

      best_val_accuracy = 0.0
      for epoch in range(self.num_epochs):
          self.model.train()
          running_loss = 0.0
          running_accuracy = 0.0
          self.dynamic_train = self.dynamic_oversamp(self.train_data, epoch)
          self.train_loader = self.create_dataloader(self.dynamic_train, self.transform)

          # monitor the process
          with tqdm(self.train_loader, unit="batch") as tepoch:
              tepoch.set_description(f"Epoch {epoch + 1}/{self.num_epochs}")

              for inputs, labels in tepoch:
                  inputs, labels = inputs.to(self.device), labels.to(self.device).long()
                  labels = labels - 1

                  #forward propagation
                  self.optimizer.zero_grad()
                  raw_outputs = self.model(inputs)
                  loss = self.criterion(raw_outputs, labels)  # loss calculation

                  loss.backward()  # backward propagation
                  self.optimizer.step()

                  #Loss calculating
                  running_loss += loss.item()
                  _, predicted = torch.max(raw_outputs, 1)
                  accuracy = (predicted == labels).sum().item()/ labels.size(0)
                  running_accuracy += accuracy
                  tepoch.set_postfix(loss=running_loss / len(self.train_loader),
                            accuracy=running_accuracy / len(self.train_loader))



          self.scheduler.step()
          print(self.scheduler.get_last_lr())


          val_loss,val_accuracy, all_labels, all_predictions = self.validate()
          print(f"Epoch {epoch + 1}/{self.num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

          if val_accuracy > best_val_accuracy:
              best_val_accuracy = val_accuracy
              torch.save(self.model.state_dict(), f'{solution_dir}/best_model.pth')
              current_time = datatime.now().strftime("%Y%m%d-%H%M")
              print('Best model saved!', current_time)

      print('Finished Training')
      print(f'Best validation accuracy: {best_val_accuracy:.4f}')



In [28]:
csv_path = './COMP90086_2024_Project_train/train.csv'
img_dir = './COMP90086_2024_Project_train/train'

In [29]:
class ContinueTrainer:
  '''
  The function here helps to initalize the parameters used in the models and pre-process the image
  '''
  def __init__(self, model_path, model, 
                csv_path, img_dir, 
                stratify_column='stable_height', 
                test_size=0.2,
                batch_size=32, num_epochs=10, learning_rate=0.001 ,random_state=42):
        self.img_dir = img_dir
        self.stratify_column = stratify_column
        self.test_size = test_size
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate

        self.model = model
        self.model_path = model_path
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.model = self.load_model()

        # load the train dataset
        self.data_frame = pd.read_csv(csv_path)

        # split data into train and validation dataset
        self.train_data, self.val_data, self.train_ids, self.valid_ids = self.split_dataset()

        hard_df = self.train_data[(self.train_data['type']==2) &
                                  (self.train_data['instability_type']==0)]

        oversample_hard = resample(hard_df,replace=True,n_samples=500,random_state=2)

        # hard_df2 = self.train_data[(self.train_data['type']==2) &
        #                           (self.train_data['instability_type']==1)]
        # oversample_hard2 = resample(hard_df2,replace=True,n_samples=300, random_state=1)

        # hard_df3 = self.train_data[(self.train_data['type']==2) &
        #                           (self.train_data['instability_type']==2)]
        # oversample_hard3 = resample(hard_df3,replace=True,n_samples=50,random_state=2)

        # simple_1_df = self.train_data[(self.train_data['type']==1) &
        #                                 (self.train_data['instability_type']==1)]
        # oversample_simpe1 = resample(simple_1_df,replace=True,n_samples=100, random_state=1)

        conca_df = pd.concat([self.train_data,oversample_hard
                        #      ,oversample_hard2, oversample_hard3,oversample_simpe1
                             ])
       # self.train_data = conca_df
        # 打乱顺序
        self.train_data = conca_df.sample(frac=1)

        # pre-processing the images
        self.transform = transforms.Compose([
            # transforms.Resize((512, 512)),

              #ADD MORE TRANSFORM METHODS HERE

            transforms.RandomHorizontalFlip(),## ADDDED
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)), ## ADDDED
            transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),## ADDDED
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


        self.train_loader = self.create_dataloader(self.train_data, self.transform)
        self.val_loader = self.create_dataloader(self.val_data, self.transform, shuffle=False)


        # set the loss weight
        '''
        Changed some thing here : to see the results
        1. add the class_weights to each class labels with different weights,
           more lables occurece with lower weight, few label occurence with higher weight;
           without the weights, all the labels will be treated equally.

        2. used to apply L2 regularization (also called weight decay).
        The primary purpose of weight decay is to prevent overfitting by penalizing large weights.
         It adds a penalty to the loss function based on the size of the weights
         helps it generalize better to unseen data.
         Lnew = Lold + weight_decay * sum(weight^2)

        3. add scheduler to Reduces the learning rate after every step_size epochs.
          After every 4 epochs, the learning rate will be multiplied by gamma (0.1 here), reducing it by 90%.
        '''
        class_weights = torch.tensor([100/25 , 100/25, 100/20,  100/15, 100/10,  100/5], device= self.device)
        self.criterion = nn.CrossEntropyLoss(weight=class_weights) # CrossEntropy for multi categorical-label predication
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate , weight_decay=0.0001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size = 10, gamma=0.15)

  def load_model(self):
    self.model.load_state_dict(torch.load(self.model_path, map_location=self.device))
    return self.model

  '''
  The function helps to split the data set into the training and validation dataset according to the
  size pre-determined.
  '''

  def split_dataset(self):
    split = StratifiedShuffleSplit(n_splits=1, test_size=self.test_size, random_state=42)
    train_ids = []
    valid_ids = []
    for train_idx, val_idx in split.split(self.data_frame, self.data_frame[self.stratify_column]):
      train_data = self.data_frame.iloc[train_idx]
      val_data = self.data_frame.iloc[val_idx]
      train_ids.append(train_idx)
      valid_ids.append(val_idx)
    print(f"Train dataset size: {len(train_data)}",
       f"Validation dataset size: {len(val_data)}",
       f"length of train_ids{(len(train_ids))}",
       f"length of valid_ids{(len(valid_ids))}")
    return train_data, val_data, train_ids, valid_ids


  '''
  The function helps to loda the image
  '''
  def create_dataloader(self, data_frame, transform, shuffle=True):
    dataset = BlockStackDataset(data_frame, self.img_dir, transform=transform) # transform 可以用来数据增强
    return DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)


  def generate_classification_report(self, outputs, labels):
    predicted = outputs # 'outputs' is already a numpy array after prediction
    labels = labels
    print(classification_report(labels, predicted, zero_division=0))



  def calculate_confusion_matrix(self, outputs, labels):
    predicted = outputs
    labels = labels
    matrix = confusion_matrix(labels, predicted)
    print(matrix)


  def validate(self):
    self.model.eval()
    val_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_labels = []  # add all the lables
    all_predictions = []  # add all the prediction

    with torch.no_grad():
        for inputs, labels in self.val_loader:
            inputs, labels = inputs.to(self.device), labels.to(self.device).long()
            labels = labels - 1
            outputs = self.model(inputs)
            predicted = torch.argmax(outputs, 1)

            # collecting all the lables and predictions
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

            # calculate the loss
            loss = self.criterion(outputs, labels)
            val_loss += loss.item()

            # calculate the correct predication
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    # calculate the accuracy rate
    val_accuracy = correct_predictions / total_samples
    self.generate_classification_report(np.array(all_predictions), np.array(all_labels))
    self.calculate_confusion_matrix(np.array(all_predictions), np.array(all_labels))
    return val_loss/len(self.val_loader),val_accuracy, all_labels, all_predictions


  '''
  The function here is used as the main training function on the image by using the pre-definned models in
  hte first model class.
  '''

  def train(self):
      _, solution_dir = create_log_dir()

      best_val_accuracy = 0.0
      for epoch in range(self.num_epochs):
          self.model.train()
          running_loss = 0.0
          running_accuracy = 0.0

          # monitor the process
          with tqdm(self.train_loader, unit="batch") as tepoch:
              tepoch.set_description(f"Epoch {epoch + 1}/{self.num_epochs}")

              for inputs, labels in tepoch:
                  inputs, labels = inputs.to(self.device), labels.to(self.device).long()
                  labels = labels - 1

                  #forward propagation
                  self.optimizer.zero_grad()
                  raw_outputs = self.model(inputs)
                  loss = self.criterion(raw_outputs, labels)  # loss calculation

                  loss.backward()  # backward propagation
                  self.optimizer.step()

                  #Loss calculating
                  running_loss += loss.item()
                  _, predicted = torch.max(raw_outputs, 1)
                  accuracy = (predicted == labels).sum().item()/ labels.size(0)
                  running_accuracy += accuracy
                  tepoch.set_postfix(loss=running_loss / len(self.train_loader),
                            accuracy=running_accuracy / len(self.train_loader))



          self.scheduler.step()
          print(self.scheduler.get_last_lr())


          val_loss,val_accuracy, all_labels, all_predictions = self.validate()
          print(f"Epoch {epoch + 1}/{self.num_epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

          if val_accuracy > best_val_accuracy:
              best_val_accuracy = val_accuracy
              torch.save(self.model.state_dict(), f'{solution_dir}/best_model.pth')
              current_time = datatime.now().strftime("%Y%m%d-%H%M")
              print('Best model saved!', current_time)

      print('Finished Training')
      print(f'Best validation accuracy: {best_val_accuracy:.4f}')



# Model with LeakyReLU with Danymic Oversampling

In [30]:
if __name__ == "__main__":
    trainer = ClassificationBlockStackTrainer1(
        model = FineTunedGoogLeNet(),
        csv_path = csv_path, 
        img_dir= img_dir, 
        test_size=0.2, 
        num_epochs=64,
        batch_size=32
        )
    trainer.train()

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1
stable_height
2    1852
1    1756
3    1470
4    1119
5     753
6     394
Name: count, dtype: int64


Epoch 1/64: 100%|██████████| 230/230 [01:14<00:00,  3.07batch/s, accuracy=0.313, loss=1.57]    


[0.001]
              precision    recall  f1-score   support

           0       0.39      0.70      0.50       384
           1       0.42      0.35      0.38       384
           2       0.42      0.22      0.29       307
           3       0.33      0.30      0.32       230
           4       0.32      0.29      0.30       154
           5       0.18      0.05      0.08        77

    accuracy                           0.38      1536
   macro avg       0.35      0.32      0.31      1536
weighted avg       0.38      0.38      0.36      1536

[[268  62   9  25  14   6]
 [185 135  16  27  16   5]
 [105  80  69  34  17   2]
 [ 71  25  41  69  23   1]
 [ 40  13  15  38  44   4]
 [ 15   9  13  14  22   4]]
Epoch 1/64, Validation Loss: 1.4369, Validation Accuracy: 0.3835
Best model saved! 20241015-0922
stable_height
2    1828
1    1754
3    1478
4    1105
5     772
6     407
Name: count, dtype: int64


Epoch 2/64: 100%|██████████| 230/230 [01:14<00:00,  3.09batch/s, accuracy=0.4, loss=1.39]     


[0.001]
              precision    recall  f1-score   support

           0       0.69      0.48      0.57       384
           1       0.60      0.38      0.46       384
           2       0.43      0.52      0.47       307
           3       0.33      0.59      0.42       230
           4       0.21      0.34      0.26       154
           5       0.33      0.01      0.03        77

    accuracy                           0.44      1536
   macro avg       0.43      0.39      0.37      1536
weighted avg       0.49      0.44      0.44      1536

[[185  41  63  51  44   0]
 [ 58 145  70  62  48   1]
 [ 20  39 159  54  34   1]
 [  6   6  60 135  23   0]
 [  1   6  10  85  52   0]
 [  0   6   5  21  44   1]]
Epoch 2/64, Validation Loss: 1.3195, Validation Accuracy: 0.4408
Best model saved! 20241015-0924
stable_height
2    1832
1    1783
3    1462
4    1107
5     766
6     394
Name: count, dtype: int64


...

[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.76      0.67      0.71       384
           1       0.76      0.72      0.74       384
           2       0.67      0.69      0.68       307
           3       0.62      0.65      0.63       230
           4       0.54      0.56      0.55       154
           5       0.35      0.52      0.42        77

    accuracy                           0.67      1536
   macro avg       0.62      0.64      0.62      1536
weighted avg       0.68      0.67      0.67      1536

[[258  38  30  25  17  16]
 [ 32 276  27  26   9  14]
 [ 16  31 213  23  15   9]
 [ 11   7  25 150  22  15]
 [ 13   6  13  15  87  20]
 [  8   5   8   4  12  40]]
Epoch 62/64, Validation Loss: 0.8507, Validation Accuracy: 0.6667
stable_height
2    1832
1    1783
3    1462
4    1107
5     766
6     394
Name: count, dtype: int64


Epoch 63/64: 100%|██████████| 230/230 [01:13<00:00,  3.11batch/s, accuracy=0.773, loss=0.625] 


[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.76      0.66      0.70       384
           1       0.74      0.70      0.72       384
           2       0.66      0.70      0.68       307
           3       0.64      0.62      0.63       230
           4       0.50      0.60      0.55       154
           5       0.35      0.51      0.42        77

    accuracy                           0.66      1536
   macro avg       0.61      0.63      0.62      1536
weighted avg       0.67      0.66      0.66      1536

[[252  38  38  21  20  15]
 [ 28 268  30  26  19  13]
 [ 21  27 216  18  12  13]
 [ 13  12  25 142  25  13]
 [ 13  10  10  12  92  17]
 [  5   6   8   4  15  39]]
Epoch 63/64, Validation Loss: 0.8709, Validation Accuracy: 0.6569
stable_height
2    1852
1    1756
3    1470
4    1119
5     753
6     394
Name: count, dtype: int64


Epoch 64/64: 100%|██████████| 230/230 [01:11<00:00,  3.21batch/s, accuracy=0.768, loss=0.623]  


[1.0000000000000002e-07]
              precision    recall  f1-score   support

           0       0.74      0.73      0.73       384
           1       0.75      0.71      0.73       384
           2       0.67      0.66      0.67       307
           3       0.65      0.65      0.65       230
           4       0.54      0.60      0.57       154
           5       0.37      0.43      0.40        77

    accuracy                           0.67      1536
   macro avg       0.62      0.63      0.62      1536
weighted avg       0.68      0.67      0.67      1536

[[279  36  21  21  21   6]
 [ 35 271  31  20  16  11]
 [ 24  26 204  25  15  13]
 [ 17  10  28 149  15  11]
 [  9  11  12  14  93  15]
 [ 14   8   7   2  13  33]]
Epoch 64/64, Validation Loss: 0.8590, Validation Accuracy: 0.6699
Finished Training
Best validation accuracy: 0.6875


# Model 
* Adjusting weight on after best model in epoch 48

In [52]:
if __name__ == "__main__":
    trainer = ContinueTrainer(
   #     model_path = './runs/20241014-oversam-epoch48-bestmodel/best_model.pth',
        model = TunnedBlockStackNet8(),
        csv_path = csv_path, 
        img_dir= img_dir, 
        test_size=0.2, 
        num_epochs=64,
        batch_size=32
        )
    trainer.train()

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/64: 100%|██████████| 222/222 [00:52<00:00,  4.19batch/s, accuracy=0.64, loss=0.937]   


[0.001]
              precision    recall  f1-score   support

           0       0.78      0.62      0.69       384
           1       0.76      0.64      0.69       384
           2       0.62      0.74      0.67       307
           3       0.54      0.66      0.59       230
           4       0.48      0.50      0.49       154
           5       0.32      0.43      0.37        77

    accuracy                           0.63      1536
   macro avg       0.58      0.60      0.58      1536
weighted avg       0.65      0.63      0.64      1536

[[237  48  40  27  16  16]
 [ 23 244  45  38  18  16]
 [ 16  13 227  29  14   8]
 [ 11   5  33 152  18  11]
 [ 12   4  13  30  77  18]
 [  3   9  11   5  16  33]]
Epoch 1/64, Validation Loss: 0.9115, Validation Accuracy: 0.6315
Best model saved! 20241014-2104


Epoch 2/64: 100%|██████████| 222/222 [00:52<00:00,  4.23batch/s, accuracy=0.64, loss=0.92]    


[0.001]
              precision    recall  f1-score   support

           0       0.71      0.70      0.70       384
           1       0.76      0.64      0.69       384
           2       0.65      0.66      0.65       307
           3       0.56      0.69      0.62       230
           4       0.47      0.62      0.54       154
           5       0.34      0.19      0.25        77

    accuracy                           0.64      1536
   macro avg       0.58      0.58      0.57      1536
weighted avg       0.64      0.64      0.64      1536

[[267  32  30  36  12   7]
 [ 51 244  36  26  21   6]
 [ 24  27 202  30  17   7]
 [ 17  10  16 158  27   2]
 [ 11   4  11  26  95   7]
 [  8   5  15   6  28  15]]
Epoch 2/64, Validation Loss: 0.9207, Validation Accuracy: 0.6387
Best model saved! 20241014-2105


...

[1.1390624999999997e-08]
              precision    recall  f1-score   support

           0       0.76      0.67      0.71       384
           1       0.73      0.68      0.71       384
           2       0.68      0.67      0.68       307
           3       0.63      0.69      0.66       230
           4       0.55      0.61      0.58       154
           5       0.31      0.44      0.37        77

    accuracy                           0.66      1536
   macro avg       0.61      0.63      0.62      1536
weighted avg       0.67      0.66      0.66      1536

[[256  40  28  26  18  16]
 [ 32 263  32  24  20  13]
 [ 20  31 207  21  13  15]
 [ 11  10  20 159  15  15]
 [ 10   8  11  15  94  16]
 [  7  10   8   6  12  34]]
Epoch 62/64, Validation Loss: 0.9797, Validation Accuracy: 0.6595


Epoch 63/64: 100%|██████████| 222/222 [00:52<00:00,  4.23batch/s, accuracy=0.792, loss=0.566]  


[1.1390624999999997e-08]
              precision    recall  f1-score   support

           0       0.77      0.66      0.71       384
           1       0.75      0.69      0.72       384
           2       0.65      0.68      0.67       307
           3       0.64      0.68      0.66       230
           4       0.55      0.62      0.58       154
           5       0.39      0.60      0.47        77

    accuracy                           0.67      1536
   macro avg       0.63      0.65      0.63      1536
weighted avg       0.68      0.67      0.67      1536

[[252  37  33  28  17  17]
 [ 30 265  33  22  23  11]
 [ 16  27 208  22  19  15]
 [ 14  11  23 156  14  12]
 [ 12   4  12  14  96  16]
 [  4   8   9   3   7  46]]
Epoch 63/64, Validation Loss: 0.9658, Validation Accuracy: 0.6660


Epoch 64/64: 100%|██████████| 222/222 [00:52<00:00,  4.24batch/s, accuracy=0.789, loss=0.579]  


[1.1390624999999997e-08]
              precision    recall  f1-score   support

           0       0.78      0.71      0.75       384
           1       0.77      0.71      0.74       384
           2       0.67      0.71      0.69       307
           3       0.67      0.67      0.67       230
           4       0.49      0.59      0.54       154
           5       0.40      0.49      0.44        77

    accuracy                           0.68      1536
   macro avg       0.63      0.65      0.64      1536
weighted avg       0.69      0.68      0.69      1536

[[274  29  33  19  19  10]
 [ 30 272  28  22  22  10]
 [ 18  26 217  16  18  12]
 [ 12  14  21 155  21   7]
 [ 12   6  13  15  91  17]
 [  5   8  10   3  13  38]]
Epoch 64/64, Validation Loss: 0.9155, Validation Accuracy: 0.6816
Finished Training
Best validation accuracy: 0.6868


In [53]:
valid1_df = trainer.val_data
_,_, all_labels, all_predictions = trainer.validate()
valid1_df['pred_label'] = [pred + 1 for pred in all_predictions]
valid1_df['pred_type'] = valid1_df['pred_label'] == valid1_df['stable_height']
valid1_df.groupby(['instability_type', 'type']).pred_type.sum()/valid1_df.groupby(['instability_type', 'type']).pred_type.count()

              precision    recall  f1-score   support

           0       0.77      0.70      0.73       384
           1       0.78      0.68      0.73       384
           2       0.65      0.70      0.68       307
           3       0.62      0.64      0.63       230
           4       0.51      0.64      0.56       154
           5       0.32      0.38      0.34        77

    accuracy                           0.66      1536
   macro avg       0.61      0.62      0.61      1536
weighted avg       0.68      0.66      0.67      1536

[[268  27  31  24  19  15]
 [ 32 262  30  26  24  10]
 [ 21  22 214  23  19   8]
 [ 12  11  29 148  17  13]
 [  9   7  10  13  98  17]
 [  6   9  13   4  16  29]]


/tmp/ipykernel_1235/2499798223.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1_df['pred_label'] = [pred + 1 for pred in all_predictions]
/tmp/ipykernel_1235/2499798223.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1_df['pred_type'] = valid1_df['pred_label'] == valid1_df['stable_height']


instability_type  type
0                 1       0.802395
                  2       0.495050
1                 1       0.671795
                  2       0.428941
2                 1       0.927184
                  2       0.902174
Name: pred_type, dtype: float64

In [69]:
print(valid1_df['instability_type'].value_counts())

instability_type
1    777
2    390
0    369
Name: count, dtype: int64


In [41]:
if __name__ == "__main__":
    trainer = ContinueTrainer(
        model_path = './runs/20241014-oversam-epoch48-bestmodel/best_model.pth',
        model = TunnedBlockStackNet8(),
        csv_path = csv_path, 
        img_dir= img_dir, 
        test_size=0.2, 
        num_epochs=8,
        batch_size=32
        )
    trainer.train()

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/8: 100%|█████████▉| 221/222 [00:51<00:00,  4.34batch/s, accuracy=0.639, loss=0.928]  /root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
Epoch 1/8: 100%|██████████| 222/222 [00:51<00:00,  4.29batch/s, accuracy=0.641, loss=0.933]


[0.001]
              precision    recall  f1-score   support

           0       0.73      0.61      0.66       384
           1       0.74      0.60      0.66       384
           2       0.69      0.64      0.66       307
           3       0.52      0.71      0.60       230
           4       0.45      0.58      0.51       154
           5       0.31      0.43      0.36        77

    accuracy                           0.62      1536
   macro avg       0.57      0.59      0.58      1536
weighted avg       0.64      0.62      0.62      1536

[[233  48  33  40  20  10]
 [ 32 230  34  40  28  20]
 [ 16  21 195  38  19  18]
 [ 10   5  13 164  25  13]
 [ 16   4   4  26  90  14]
 [ 13   2   3   6  20  33]]
Epoch 1/8, Validation Loss: 0.9815, Validation Accuracy: 0.6152
Best model saved! 20241014-2034


Epoch 2/8: 100%|██████████| 222/222 [00:52<00:00,  4.21batch/s, accuracy=0.638, loss=0.929]  


[0.001]
              precision    recall  f1-score   support

           0       0.69      0.72      0.70       384
           1       0.75      0.67      0.71       384
           2       0.61      0.67      0.64       307
           3       0.59      0.60      0.60       230
           4       0.50      0.52      0.51       154
           5       0.37      0.31      0.34        77

    accuracy                           0.64      1536
   macro avg       0.59      0.58      0.58      1536
weighted avg       0.64      0.64      0.64      1536

[[275  32  31  24  14   8]
 [ 44 256  33  24  17  10]
 [ 39  25 206  22  11   4]
 [ 20  12  30 138  22   8]
 [ 15   7  22  19  80  11]
 [  8   9  14   6  16  24]]
Epoch 2/8, Validation Loss: 0.9350, Validation Accuracy: 0.6374
Best model saved! 20241014-2035


...

[0.001]
              precision    recall  f1-score   support

           0       0.67      0.72      0.69       384
           1       0.69      0.71      0.70       384
           2       0.65      0.60      0.62       307
           3       0.59      0.53      0.56       230
           4       0.52      0.42      0.46       154
           5       0.28      0.43      0.34        77

    accuracy                           0.62      1536
   macro avg       0.57      0.57      0.56      1536
weighted avg       0.62      0.62      0.62      1536

[[275  42  31  17  11   8]
 [ 37 271  33  20  10  13]
 [ 27  39 185  34   8  14]
 [ 27  16  15 122  28  22]
 [ 23  15  14   8  64  30]
 [ 19  11   7   5   2  33]]
Epoch 6/8, Validation Loss: 0.9817, Validation Accuracy: 0.6185


Epoch 7/8: 100%|██████████| 222/222 [00:52<00:00,  4.21batch/s, accuracy=0.645, loss=0.918]  


[0.001]
              precision    recall  f1-score   support

           0       0.82      0.54      0.66       384
           1       0.66      0.77      0.71       384
           2       0.59      0.69      0.64       307
           3       0.60      0.54      0.57       230
           4       0.53      0.61      0.57       154
           5       0.31      0.36      0.33        77

    accuracy                           0.63      1536
   macro avg       0.58      0.59      0.58      1536
weighted avg       0.65      0.63      0.63      1536

[[209  62  45  35  16  17]
 [ 15 294  28  26  15   6]
 [ 10  42 213  14  17  11]
 [ 11  18  40 124  25  12]
 [  6  14  16   7  94  17]
 [  3  14  19   2  11  28]]
Epoch 7/8, Validation Loss: 0.9344, Validation Accuracy: 0.6263


Epoch 8/8: 100%|██████████| 222/222 [00:51<00:00,  4.29batch/s, accuracy=0.651, loss=0.896]  


[0.001]
              precision    recall  f1-score   support

           0       0.84      0.58      0.69       384
           1       0.75      0.68      0.71       384
           2       0.67      0.65      0.66       307
           3       0.45      0.78      0.58       230
           4       0.37      0.39      0.38       154
           5       0.35      0.31      0.33        77

    accuracy                           0.62      1536
   macro avg       0.57      0.57      0.56      1536
weighted avg       0.65      0.62      0.62      1536

[[222  46  38  47  20  11]
 [ 15 261  29  44  25  10]
 [  7  31 200  42  15  12]
 [  9   2  18 180  16   5]
 [  6   6   9  67  60   6]
 [  5   4   3  16  25  24]]
Epoch 8/8, Validation Loss: 0.9748, Validation Accuracy: 0.6165
Finished Training
Best validation accuracy: 0.6393


In [42]:
valid1_df = trainer.val_data
_,_, all_labels, all_predictions = trainer.validate()
valid1_df['pred_label'] = [pred + 1 for pred in all_predictions]
valid1_df['pred_type'] = valid1_df['pred_label'] == valid1_df['stable_height']
valid1_df.groupby(['instability_type', 'type']).pred_type.sum()/valid1_df.groupby(['instability_type', 'type']).pred_type.count()

              precision    recall  f1-score   support

           0       0.79      0.58      0.67       384
           1       0.74      0.67      0.70       384
           2       0.66      0.64      0.65       307
           3       0.44      0.74      0.56       230
           4       0.42      0.40      0.41       154
           5       0.39      0.38      0.38        77

    accuracy                           0.61      1536
   macro avg       0.57      0.57      0.56      1536
weighted avg       0.64      0.61      0.62      1536

[[223  45  41  51  15   9]
 [ 22 256  27  45  23  11]
 [ 10  31 198  44  13  11]
 [ 15   3  21 171  11   9]
 [  9   5   9  63  62   6]
 [  5   4   3  12  24  29]]


/tmp/ipykernel_1235/2499798223.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1_df['pred_label'] = [pred + 1 for pred in all_predictions]
/tmp/ipykernel_1235/2499798223.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1_df['pred_type'] = valid1_df['pred_label'] == valid1_df['stable_height']


instability_type  type
0                 1       0.718563
                  2       0.460396
1                 1       0.625641
                  2       0.366925
2                 1       0.873786
                  2       0.869565
Name: pred_type, dtype: float64

# Add hard example 
* Instability type 0 1 2 random sampling with ratio (500, 400, 50)
* Acc: 0.68

In [34]:
# 
if __name__ == "__main__":
    trainer = ContinueTrainer(
        model_path = './runs/20241014-oversam_instab1-epoch16/best_model.pth',
        model = TunnedBlockStackNet8(),
        csv_path = csv_path, 
        img_dir= img_dir, 
        test_size=0.2, 
        num_epochs=32,
        batch_size=32
        )
    trainer.train()

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/32: 100%|█████████▉| 218/219 [00:50<00:00,  4.15batch/s, accuracy=0.566, loss=1.08]   /root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
Epoch 1/32: 100%|██████████| 219/219 [00:51<00:00,  4.28batch/s, accuracy=0.568, loss=1.09]


[0.001]
              precision    recall  f1-score   support

           0       0.70      0.61      0.66       384
           1       0.61      0.74      0.67       384
           2       0.66      0.59      0.62       307
           3       0.55      0.35      0.43       230
           4       0.34      0.44      0.39       154
           5       0.20      0.31      0.25        77

    accuracy                           0.57      1536
   macro avg       0.51      0.51      0.50      1536
weighted avg       0.59      0.57      0.57      1536

[[236  68  28  21  21  10]
 [ 20 283  37  15  11  18]
 [ 17  48 182  27  19  14]
 [ 26  24  14  80  69  17]
 [ 21  17  10   3  68  35]
 [ 16  22   5   0  10  24]]
Epoch 1/32, Validation Loss: 1.0438, Validation Accuracy: 0.5684
Best model saved! 20241014-1947


Epoch 2/32: 100%|██████████| 219/219 [00:50<00:00,  4.31batch/s, accuracy=0.58, loss=1.06]    


[0.001]
              precision    recall  f1-score   support

           0       0.63      0.64      0.63       384
           1       0.68      0.65      0.67       384
           2       0.56      0.73      0.63       307
           3       0.56      0.45      0.50       230
           4       0.47      0.29      0.36       154
           5       0.28      0.38      0.32        77

    accuracy                           0.58      1536
   macro avg       0.53      0.52      0.52      1536
weighted avg       0.59      0.58      0.58      1536

[[244  50  55  26   5   4]
 [ 35 251  59  23   7   9]
 [ 20  23 224  23   8   9]
 [ 35  17  28 103  25  22]
 [ 35  18  19   7  44  31]
 [ 17  11  15   1   4  29]]
Epoch 2/32, Validation Loss: 1.0425, Validation Accuracy: 0.5827
Best model saved! 20241014-1948


...

[3.3749999999999995e-06]
              precision    recall  f1-score   support

           0       0.76      0.65      0.70       384
           1       0.77      0.67      0.72       384
           2       0.65      0.66      0.65       307
           3       0.57      0.67      0.61       230
           4       0.51      0.63      0.56       154
           5       0.35      0.45      0.40        77

    accuracy                           0.65      1536
   macro avg       0.60      0.62      0.61      1536
weighted avg       0.67      0.65      0.65      1536

[[250  36  34  34  17  13]
 [ 27 258  30  37  21  11]
 [ 22  27 202  23  19  14]
 [ 11   7  22 153  27  10]
 [ 10   3  10  18  97  16]
 [  9   5  13   4  11  35]]
Epoch 30/32, Validation Loss: 0.8871, Validation Accuracy: 0.6478


Epoch 31/32: 100%|██████████| 219/219 [00:51<00:00,  4.28batch/s, accuracy=0.732, loss=0.703]  


[3.3749999999999995e-06]
              precision    recall  f1-score   support

           0       0.75      0.67      0.71       384
           1       0.77      0.67      0.72       384
           2       0.68      0.66      0.67       307
           3       0.61      0.63      0.62       230
           4       0.51      0.65      0.57       154
           5       0.31      0.49      0.38        77

    accuracy                           0.65      1536
   macro avg       0.60      0.63      0.61      1536
weighted avg       0.67      0.65      0.66      1536

[[258  42  31  25  11  17]
 [ 28 258  29  31  22  16]
 [ 18  23 204  25  21  16]
 [ 19   1  23 145  27  15]
 [  8   6   9   9 100  22]
 [ 11   6   6   2  14  38]]
Epoch 31/32, Validation Loss: 0.8837, Validation Accuracy: 0.6530


Epoch 32/32: 100%|██████████| 219/219 [00:50<00:00,  4.30batch/s, accuracy=0.737, loss=0.693]  


[3.3749999999999995e-06]
              precision    recall  f1-score   support

           0       0.75      0.66      0.70       384
           1       0.78      0.68      0.73       384
           2       0.65      0.71      0.68       307
           3       0.59      0.63      0.61       230
           4       0.53      0.62      0.57       154
           5       0.30      0.42      0.35        77

    accuracy                           0.65      1536
   macro avg       0.60      0.62      0.61      1536
weighted avg       0.67      0.65      0.66      1536

[[252  35  34  32  14  17]
 [ 26 262  35  24  25  12]
 [ 17  17 218  28  16  11]
 [ 18   8  27 146  17  14]
 [ 10   7  10  11  95  21]
 [ 11   6  11   5  12  32]]
Epoch 32/32, Validation Loss: 0.8889, Validation Accuracy: 0.6543
Finished Training
Best validation accuracy: 0.6803


In [35]:
valid1_df = trainer.val_data
_,_, all_labels, all_predictions = trainer.validate()
valid1_df['pred_label'] = [pred + 1 for pred in all_predictions]
valid1_df['pred_type'] = valid1_df['pred_label'] == valid1_df['stable_height']

              precision    recall  f1-score   support

           0       0.75      0.67      0.71       384
           1       0.77      0.69      0.73       384
           2       0.68      0.68      0.68       307
           3       0.58      0.64      0.61       230
           4       0.50      0.58      0.53       154
           5       0.30      0.40      0.34        77

    accuracy                           0.65      1536
   macro avg       0.60      0.61      0.60      1536
weighted avg       0.67      0.65      0.66      1536

[[257  33  28  39  14  13]
 [ 34 266  23  29  21  11]
 [ 18  26 210  21  20  12]
 [ 17   4  26 148  21  14]
 [  9   7  11  16  89  22]
 [  6  11  11   4  14  31]]


/tmp/ipykernel_1235/2315130222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1_df['pred_label'] = [pred + 1 for pred in all_predictions]
/tmp/ipykernel_1235/2315130222.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid1_df['pred_type'] = valid1_df['pred_label'] == valid1_df['stable_height']


In [37]:
valid1_df.groupby(['instability_type', 'type']).pred_type.sum()/valid1_df.groupby(['instability_type', 'type']).pred_type.count()

instability_type  type
0                 1       0.778443
                  2       0.554455
1                 1       0.648718
                  2       0.377261
2                 1       0.941748
                  2       0.902174
Name: pred_type, dtype: float64

# Add hard example 
* Instability type 0 1 2 random sampling with ratio (500, 200, 50)
* Acc: 0.67

In [22]:
# 
if __name__ == "__main__":
    trainer2 = ContinueTrainer(
        model_path = './runs/20241014-oversam_instab1-epoch16/best_model.pth',
        model = TunnedBlockStackNet8(),
        csv_path = csv_path, 
        img_dir= img_dir, 
        test_size=0.2, 
        num_epochs=32,
        batch_size=32
        )
    trainer2.train()
        

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/32:   0%|          | 0/216 [00:00<?, ?batch/s]/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
Epoch 1/32: 100%|██████████| 216/216 [00:53<00:00,  4.07batch/s, accuracy=0.571, loss=1.08]  


[0.001]
              precision    recall  f1-score   support

           0       0.71      0.56      0.63       384
           1       0.78      0.57      0.66       384
           2       0.62      0.65      0.63       307
           3       0.52      0.59      0.55       230
           4       0.37      0.44      0.40       154
           5       0.27      0.64      0.38        77

    accuracy                           0.58      1536
   macro avg       0.54      0.57      0.54      1536
weighted avg       0.62      0.58      0.59      1536

[[216  32  51  36  24  25]
 [ 46 220  36  30  19  33]
 [ 13  19 200  27  24  24]
 [ 15   4  28 135  28  20]
 [ 14   4   7  30  67  32]
 [  1   3   2   2  20  49]]
Epoch 1/32, Validation Loss: 1.0249, Validation Accuracy: 0.5775
Best model saved! 20241014-1904


Epoch 2/32: 100%|██████████| 216/216 [00:50<00:00,  4.28batch/s, accuracy=0.565, loss=1.07]  


[0.001]
              precision    recall  f1-score   support

           0       0.70      0.65      0.67       384
           1       0.68      0.68      0.68       384
           2       0.67      0.64      0.65       307
           3       0.57      0.59      0.58       230
           4       0.42      0.57      0.48       154
           5       0.24      0.17      0.20        77

    accuracy                           0.61      1536
   macro avg       0.54      0.55      0.54      1536
weighted avg       0.62      0.61      0.61      1536

[[248  49  38  24  22   3]
 [ 38 260  31  33  15   7]
 [ 14  32 197  27  27  10]
 [ 24  17  16 135  29   9]
 [ 15  14   7  17  88  13]
 [ 14  13   6   1  30  13]]
Epoch 2/32, Validation Loss: 1.0111, Validation Accuracy: 0.6126
Best model saved! 20241014-1905


...

[3.3749999999999995e-06]
              precision    recall  f1-score   support

           0       0.74      0.67      0.70       384
           1       0.75      0.70      0.72       384
           2       0.68      0.69      0.69       307
           3       0.64      0.65      0.64       230
           4       0.48      0.59      0.53       154
           5       0.33      0.43      0.37        77

    accuracy                           0.66      1536
   macro avg       0.60      0.62      0.61      1536
weighted avg       0.67      0.66      0.66      1536

[[256  37  34  23  20  14]
 [ 28 268  27  26  21  14]
 [ 18  28 213  20  17  11]
 [ 20   6  20 149  23  12]
 [ 13   9  11  14  91  16]
 [  9   9   8   1  17  33]]
Epoch 30/32, Validation Loss: 0.8882, Validation Accuracy: 0.6576


Epoch 31/32: 100%|██████████| 216/216 [00:49<00:00,  4.34batch/s, accuracy=0.734, loss=0.706]  


[3.3749999999999995e-06]
              precision    recall  f1-score   support

           0       0.76      0.67      0.71       384
           1       0.73      0.71      0.72       384
           2       0.69      0.70      0.70       307
           3       0.65      0.63      0.64       230
           4       0.49      0.62      0.55       154
           5       0.33      0.44      0.38        77

    accuracy                           0.66      1536
   macro avg       0.61      0.63      0.62      1536
weighted avg       0.68      0.66      0.67      1536

[[256  45  27  24  20  12]
 [ 25 273  30  24  20  12]
 [ 19  26 215  16  21  10]
 [ 17  12  20 144  24  13]
 [ 12   8  10   8  95  21]
 [  6  11   8   4  14  34]]
Epoch 31/32, Validation Loss: 0.8623, Validation Accuracy: 0.6621


Epoch 32/32: 100%|██████████| 216/216 [00:49<00:00,  4.36batch/s, accuracy=0.733, loss=0.714]  


[3.3749999999999995e-06]
              precision    recall  f1-score   support

           0       0.74      0.67      0.70       384
           1       0.77      0.68      0.72       384
           2       0.69      0.67      0.68       307
           3       0.64      0.65      0.64       230
           4       0.49      0.64      0.56       154
           5       0.34      0.49      0.40        77

    accuracy                           0.66      1536
   macro avg       0.61      0.63      0.62      1536
weighted avg       0.68      0.66      0.66      1536

[[257  36  31  25  22  13]
 [ 34 262  31  26  19  12]
 [ 20  25 207  19  21  15]
 [ 19   8  17 150  21  15]
 [ 10   3  11  14  98  18]
 [  7   8   4   2  18  38]]
Epoch 32/32, Validation Loss: 0.8856, Validation Accuracy: 0.6589
Finished Training
Best validation accuracy: 0.6758


In [27]:
valid_df = trainer2.val_data
_,_, all_labels, all_predictions = trainer2.validate()

              precision    recall  f1-score   support

           0       0.73      0.68      0.71       384
           1       0.77      0.69      0.73       384
           2       0.70      0.70      0.70       307
           3       0.60      0.62      0.61       230
           4       0.49      0.59      0.53       154
           5       0.35      0.47      0.40        77

    accuracy                           0.66      1536
   macro avg       0.61      0.62      0.61      1536
weighted avg       0.67      0.66      0.66      1536

[[263  32  28  29  15  17]
 [ 36 264  31  26  19   8]
 [ 20  22 214  19  20  12]
 [ 20   8  20 143  28  11]
 [ 15   5   9  15  91  19]
 [  7  10   5   5  14  36]]


In [31]:
valid_df['pred_label'] = [pred + 1 for pred in all_predictions]
valid_df['pred_type'] = valid_df['pred_label'] == valid_df['stable_height']

/tmp/ipykernel_1235/3749961525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['pred_label'] = [pred + 1 for pred in all_predictions]
/tmp/ipykernel_1235/3749961525.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['pred_type'] = valid_df['pred_label'] == valid_df['stable_height']


In [36]:
valid_df.groupby(['instability_type', 'type']).pred_type.sum()/valid_df.groupby(['instability_type', 'type']).pred_type.count()

instability_type  type
0                 1       0.826347
                  2       0.500000
1                 1       0.687179
                  2       0.361757
2                 1       0.922330
                  2       0.945652
Name: pred_type, dtype: float64

### Model with oversampling hard examples in instability type 1
* add 500 hard examples in instability type 1
* need increase epoches

In [16]:
# test
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=16,
        batch_size=32
        )
    trainer.train()
        

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/16: 100%|██████████| 208/208 [06:05<00:00,  1.76s/batch, accuracy=0.284, loss=1.57]    


[0.001]
              precision    recall  f1-score   support

           0       0.61      0.33      0.43       384
           1       0.32      0.75      0.44       384
           2       0.25      0.26      0.25       307
           3       0.18      0.07      0.11       230
           4       0.29      0.01      0.02       154
           5       0.00      0.00      0.00        77

    accuracy                           0.33      1536
   macro avg       0.27      0.24      0.21      1536
weighted avg       0.34      0.33      0.29      1536

[[128 195  48  13   0   0]
 [ 40 287  42  13   2   0]
 [ 11 203  79  13   1   0]
 [ 14 113  85  17   1   0]
 [ 12  72  46  22   2   0]
 [  4  40  17  15   1   0]]
Epoch 1/16, Validation Loss: 1.8216, Validation Accuracy: 0.3340
Best model saved! 20241014-1742


Epoch 2/16: 100%|██████████| 208/208 [06:03<00:00,  1.75s/batch, accuracy=0.371, loss=1.46]   


[0.001]
              precision    recall  f1-score   support

           0       0.53      0.51      0.52       384
           1       0.47      0.41      0.44       384
           2       0.32      0.46      0.38       307
           3       0.19      0.26      0.22       230
           4       0.17      0.07      0.10       154
           5       0.21      0.04      0.07        77

    accuracy                           0.37      1536
   macro avg       0.32      0.29      0.29      1536
weighted avg       0.37      0.37      0.36      1536

[[196  59  69  44  13   3]
 [ 82 159  81  54   8   0]
 [ 37  83 141  40   5   1]
 [ 22  27 108  59  10   4]
 [ 24   8  31  77  11   3]
 [ 10   5  14  29  16   3]]
Epoch 2/16, Validation Loss: 1.4574, Validation Accuracy: 0.3704
Best model saved! 20241014-1749


...

[0.001]
              precision    recall  f1-score   support

           0       0.57      0.69      0.62       384
           1       0.66      0.67      0.66       384
           2       0.61      0.54      0.57       307
           3       0.54      0.48      0.51       230
           4       0.39      0.47      0.42       154
           5       0.28      0.06      0.11        77

    accuracy                           0.57      1536
   macro avg       0.51      0.48      0.48      1536
weighted avg       0.56      0.57      0.56      1536

[[264  58  34  18   8   2]
 [ 45 256  37  20  24   2]
 [ 46  33 166  38  21   3]
 [ 49  20  12 111  38   0]
 [ 37  11   9  19  72   6]
 [ 22  11  15   0  24   5]]
Epoch 14/16, Validation Loss: 1.1130, Validation Accuracy: 0.5690


Epoch 15/16: 100%|██████████| 208/208 [06:11<00:00,  1.79s/batch, accuracy=0.569, loss=1.08]  


[0.0001]
              precision    recall  f1-score   support

           0       0.75      0.66      0.70       384
           1       0.69      0.60      0.64       384
           2       0.61      0.62      0.61       307
           3       0.65      0.44      0.52       230
           4       0.34      0.66      0.45       154
           5       0.30      0.34      0.32        77

    accuracy                           0.59      1536
   macro avg       0.55      0.55      0.54      1536
weighted avg       0.62      0.59      0.60      1536

[[252  40  34  17  31  10]
 [ 38 231  41  17  39  18]
 [ 17  29 191  17  40  13]
 [ 17  22  26 101  58   6]
 [ 12  10  13   4 102  13]
 [  2   5  10   0  34  26]]
Epoch 15/16, Validation Loss: 1.0438, Validation Accuracy: 0.5879
Best model saved! 20241014-1920


Epoch 16/16: 100%|██████████| 208/208 [06:09<00:00,  1.78s/batch, accuracy=0.607, loss=0.991] 


[0.0001]
              precision    recall  f1-score   support

           0       0.83      0.59      0.69       384
           1       0.72      0.65      0.68       384
           2       0.59      0.68      0.63       307
           3       0.57      0.57      0.57       230
           4       0.41      0.58      0.48       154
           5       0.28      0.44      0.34        77

    accuracy                           0.61      1536
   macro avg       0.57      0.58      0.57      1536
weighted avg       0.65      0.61      0.62      1536

[[225  45  42  33  27  12]
 [ 15 249  51  26  19  24]
 [ 10  24 209  27  24  13]
 [  7  14  23 130  41  15]
 [ 10   7  15   9  89  24]
 [  5   5  13   2  18  34]]
Epoch 16/16, Validation Loss: 0.9651, Validation Accuracy: 0.6094
Best model saved! 20241014-1927
Finished Training
Best validation accuracy: 0.6094


In [18]:
valid_ds = trainer.val_data
_,val_accuracy, all_labels, all_predictions = trainer.validate()
valid_ds['predicted'] = all_predictions
valid_ds.head()

              precision    recall  f1-score   support

           0       0.78      0.56      0.65       384
           1       0.71      0.66      0.68       384
           2       0.60      0.69      0.64       307
           3       0.55      0.53      0.54       230
           4       0.43      0.58      0.50       154
           5       0.23      0.36      0.28        77

    accuracy                           0.60      1536
   macro avg       0.55      0.56      0.55      1536
weighted avg       0.63      0.60      0.61      1536

[[214  50  44  35  24  17]
 [ 18 253  48  24  22  19]
 [ 13  24 212  28  16  14]
 [ 11  14  24 121  44  16]
 [ 11   4  13   9  90  27]
 [  9  11  14   2  13  28]]


/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_95107/284521822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_ds['predicted'] = all_predictions


,id,shapeset,type,total_height,instability_type,cam_angle,stable_height,predicted
7334,956915,2,2,4,2,1,1,0
3955,516709,1,2,5,1,1,3,1
619,77447,1,2,5,1,1,3,2
1594,212770,2,2,4,2,1,1,0
5645,745098,2,2,6,1,1,4,4


In [20]:
valid_ds['predicted'] = valid_ds['predicted'] + 1
valid_ds['pred_type'] = valid_ds['predicted'] == valid_ds['stable_height']

valid_ds.pred_type.sum()/len(valid_ds)

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_95107/911565123.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_ds['predicted'] = valid_ds['predicted'] + 1
/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_95107/911565123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_ds['pred_type'] = valid_ds['predicted'] == valid_ds['stable_height']


0.59765625

In [21]:
valid_ds.groupby(['instability_type', 'type']).pred_type.sum()/valid_ds.groupby(['instability_type', 'type']).pred_type.count()

instability_type  type
0                 1       0.796407
                  2       0.608911
1                 1       0.500000
                  2       0.299742
2                 1       0.898058
                  2       0.902174
Name: pred_type, dtype: float64

In [9]:
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=16,
        batch_size=32
        )
    trainer.train()
        

Train dataset size: 6144 Validation dataset size: 1536 length of train_ids1 length of valid_ids1


Epoch 1/16: 100%|██████████| 192/192 [06:25<00:00,  2.01s/batch, accuracy=0.621, loss=0.814]  


[0.001]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       369
           1       0.63      0.99      0.77       777
           2       0.98      0.78      0.87       390

    accuracy                           0.70      1536
   macro avg       0.54      0.59      0.55      1536
weighted avg       0.57      0.70      0.61      1536

[[  0 368   1]
 [  0 773   4]
 [  0  87 303]]
Epoch 1/16, Validation Loss: 0.6589, Validation Accuracy: 0.7005
Best model saved!


Epoch 2/16: 100%|██████████| 192/192 [06:23<00:00,  2.00s/batch, accuracy=0.689, loss=0.664]  


[0.001]
              precision    recall  f1-score   support

           0       0.67      0.05      0.09       369
           1       0.65      0.97      0.78       777
           2       0.93      0.83      0.87       390

    accuracy                           0.71      1536
   macro avg       0.75      0.62      0.58      1536
weighted avg       0.72      0.71      0.64      1536

[[ 18 340  11]
 [  8 754  15]
 [  1  66 323]]
Epoch 2/16, Validation Loss: 0.6186, Validation Accuracy: 0.7129
Best model saved!


...

[0.001]
              precision    recall  f1-score   support

           0       0.54      0.47      0.50       369
           1       0.75      0.82      0.78       777
           2       0.97      0.90      0.93       390

    accuracy                           0.76      1536
   macro avg       0.75      0.73      0.74      1536
weighted avg       0.75      0.76      0.75      1536

[[172 192   5]
 [134 637   6]
 [ 13  26 351]]
Epoch 14/16, Validation Loss: 0.5306, Validation Accuracy: 0.7552


Epoch 15/16: 100%|██████████| 192/192 [06:20<00:00,  1.98s/batch, accuracy=0.775, loss=0.517]  


[0.0001]
              precision    recall  f1-score   support

           0       0.63      0.37      0.47       369
           1       0.73      0.90      0.81       777
           2       0.97      0.90      0.93       390

    accuracy                           0.77      1536
   macro avg       0.78      0.72      0.74      1536
weighted avg       0.77      0.77      0.76      1536

[[137 225   7]
 [ 75 697   5]
 [  6  32 352]]
Epoch 15/16, Validation Loss: 0.5107, Validation Accuracy: 0.7721


Epoch 16/16: 100%|██████████| 192/192 [06:20<00:00,  1.98s/batch, accuracy=0.786, loss=0.48]   


[0.0001]
              precision    recall  f1-score   support

           0       0.59      0.41      0.48       369
           1       0.74      0.86      0.79       777
           2       0.96      0.93      0.94       390

    accuracy                           0.77      1536
   macro avg       0.76      0.73      0.74      1536
weighted avg       0.76      0.77      0.76      1536

[[151 212   6]
 [101 666  10]
 [  5  22 363]]
Epoch 16/16, Validation Loss: 0.4951, Validation Accuracy: 0.7682
Finished Training
Best validation accuracy: 0.7754


In [23]:
train_ids = trainer.train_ids
valid_ids = trainer.valid_ids

NameError: name 'trainer' is not defined

In [12]:
val_loss,val_accuracy, all_labels, all_predictions = trainer.validate()

              precision    recall  f1-score   support

           0       0.62      0.41      0.49       369
           1       0.75      0.88      0.81       777
           2       0.97      0.93      0.95       390

    accuracy                           0.78      1536
   macro avg       0.78      0.74      0.75      1536
weighted avg       0.77      0.78      0.77      1536

[[152 211   6]
 [ 88 683   6]
 [  7  19 364]]


In [13]:
comparison = map(lambda all_labels, all_predictions: all_labels == all_predictions, all_labels, all_predictions)
sum(comparison)/len(all_labels)

0.7805989583333334

In [14]:
val_data = trainer.val_data
val_data.head()

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height
7334,956915,2,2,4,2,1,1
3955,516709,1,2,5,1,1,3
619,77447,1,2,5,1,1,3
1594,212770,2,2,4,2,1,1
5645,745098,2,2,6,1,1,4


In [15]:
val_data['predicted'] = all_predictions
val_data.head()

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_37333/1695572929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['predicted'] = all_predictions


,id,shapeset,type,total_height,instability_type,cam_angle,stable_height,predicted
7334,956915,2,2,4,2,1,1,2
3955,516709,1,2,5,1,1,3,1
619,77447,1,2,5,1,1,3,1
1594,212770,2,2,4,2,1,1,2
5645,745098,2,2,6,1,1,4,1


In [16]:
val_data['pred_type'] = val_data['predicted'] == val_data['instability_type']
val_data.head()

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_37333/3086623943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['pred_type'] = val_data['predicted'] == val_data['instability_type']


,id,shapeset,type,total_height,instability_type,cam_angle,stable_height,predicted,pred_type
7334,956915,2,2,4,2,1,1,2,True
3955,516709,1,2,5,1,1,3,1,True
619,77447,1,2,5,1,1,3,1,True
1594,212770,2,2,4,2,1,1,2,True
5645,745098,2,2,6,1,1,4,1,True


In [18]:
len(val_data[val_data['predicted']== val_data['instability_type']])/len(valid_ids[0])

0.7805989583333334

In [88]:
val_data['pred_type'].sum()/len(val_data)

0.64453125

In [91]:
val_data.groupby('instability_type').pred_type.count()

instability_type
0    369
1    777
2    390
Name: pred_type, dtype: int64

In [22]:
val_data.groupby('shapeset').pred_type.sum()/val_data.groupby('shapeset').pred_type.count()

shapeset
1    0.812734
2    0.763473
Name: pred_type, dtype: float64

In [21]:
val_data.groupby('type').pred_type.sum()/val_data.groupby('type').pred_type.count()

type
1    0.821756
2    0.739974
Name: pred_type, dtype: float64

In [20]:
val_data.groupby('instability_type').pred_type.sum()/val_data.groupby('instability_type').pred_type.count()

instability_type
0    0.411924
1    0.879022
2    0.933333
Name: pred_type, dtype: float64

In [23]:
val_data.groupby('cam_angle').pred_type.sum()/val_data.groupby('cam_angle').pred_type.count()

cam_angle
1    0.802426
2    0.714660
Name: pred_type, dtype: float64

In [3]:
import pandas as pd
df = pd.read_csv('./COMP90086_2024_Project_train/train.csv')
df.head()

,id,shapeset,type,total_height,instability_type,cam_angle,stable_height
0,54,2,1,3,1,1,2
1,173,1,1,4,1,2,1
2,245,1,1,4,1,2,1
3,465,2,1,5,0,1,5
4,611,2,1,3,1,1,1


In [6]:
df.groupby('instability_type').count()

,id,shapeset,type,total_height,cam_angle,stable_height
instability_type,,,,,,
0,1920,1920,1920,1920,1920,1920
1,3840,3840,3840,3840,3840,3840
2,1920,1920,1920,1920,1920,1920


In [10]:
best_model = torch.load('./runs/experiment_20241012-010838/best_model.pth')

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_67928/1284981419.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load('./runs/experiment_20

In [15]:
best_model = torch.load('./runs/experiment_20241012-010838/best_model.pth',map_location=torch.device('cpu'))
best_model.eval()

/var/folders/gk/b497571d49ndglvgyk5sb53h0000gn/T/ipykernel_67928/1886415100.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load('./runs/experiment_20

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

# Load Model

Test Model

Net8, epoch = 8, no batch_size defined

In [13]:
if __name__ == "__main__":
    model = TunnedBlockStackNet8()
    trainer = ClassificationBlockStackTrainer1(
        # csv_file = '/content/drive/MyDrive/CV final project/train data/train.csv', ##
        # img_dir='/content/drive/MyDrive/CV final project/train data/train', ##
        csv_file = './COMP90086_2024_Project_train/train.csv', ##
        img_dir='./COMP90086_2024_Project_train/train', ##
        model=model,
        test_size=0.2, # used to control the size of data in split_dataset(self)
        num_epochs=8,
        #batch_size=32
        )
    trainer.train()

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /Users/macbookpro/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:03<00:00, 14.0MB/s]


Train dataset size: 6144 Validation dataset size: 1536


Epoch 1/8: 100%|██████████| 192/192 [06:38<00:00,  2.07s/batch, accuracy=0.274, loss=1.58]   


[0.001]
              precision    recall  f1-score   support

           0       0.49      0.42      0.45       384
           1       0.38      0.45      0.41       384
           2       0.30      0.62      0.41       307
           3       0.25      0.06      0.09       230
           4       0.26      0.12      0.16       154
           5       0.00      0.00      0.00        77

    accuracy                           0.36      1536
   macro avg       0.28      0.28      0.25      1536
weighted avg       0.34      0.36      0.33      1536

[[161 134  78   4   7   0]
 [ 70 172 132   4   6   0]
 [ 26  71 191  12   7   0]
 [ 28  47 124  13  18   0]
 [ 32  12  76  16  18   0]
 [ 13  11  35   4  14   0]]
Epoch 1/8, Validation Loss: 1.5266, Validation Accuracy: 0.3613
Best model saved!


Epoch 2/8: 100%|██████████| 192/192 [06:44<00:00,  2.11s/batch, accuracy=0.363, loss=1.45]   


...

[0.001]
              precision    recall  f1-score   support

           0       0.77      0.38      0.51       384
           1       0.69      0.51      0.58       384
           2       0.53      0.53      0.53       307
           3       0.42      0.47      0.44       230
           4       0.25      0.69      0.37       154
           5       0.12      0.12      0.12        77

    accuracy                           0.47      1536
   macro avg       0.46      0.45      0.43      1536
weighted avg       0.56      0.47      0.49      1536

[[147  52  53  45  80   7]
 [ 23 194  65  32  46  24]
 [  6  17 162  56  51  15]
 [  9   5  19 109  84   4]
 [  6   6   6  17 106  13]
 [  1   8   0   1  58   9]]
Epoch 6/8, Validation Loss: 1.3388, Validation Accuracy: 0.4733


Epoch 7/8: 100%|██████████| 192/192 [06:46<00:00,  2.12s/batch, accuracy=0.49, loss=1.22]    


[0.001]
              precision    recall  f1-score   support

           0       0.44      0.70      0.54       384
           1       0.51      0.58      0.54       384
           2       0.48      0.23      0.31       307
           3       0.42      0.33      0.37       230
           4       0.40      0.38      0.39       154
           5       0.31      0.05      0.09        77

    accuracy                           0.46      1536
   macro avg       0.43      0.38      0.37      1536
weighted avg       0.45      0.46      0.43      1536

[[268  78  14  18   6   0]
 [103 222  22  20  15   2]
 [ 72 101  71  47  14   2]
 [ 66  20  33  76  35   0]
 [ 63   7   5  15  59   5]
 [ 38   9   4   3  19   4]]
Epoch 7/8, Validation Loss: 1.3688, Validation Accuracy: 0.4557


Epoch 8/8: 100%|██████████| 192/192 [06:54<00:00,  2.16s/batch, accuracy=0.515, loss=1.2]    


[0.001]
              precision    recall  f1-score   support

           0       0.64      0.58      0.61       384
           1       0.60      0.60      0.60       384
           2       0.48      0.57      0.52       307
           3       0.47      0.50      0.49       230
           4       0.36      0.36      0.36       154
           5       0.26      0.13      0.17        77

    accuracy                           0.53      1536
   macro avg       0.47      0.46      0.46      1536
weighted avg       0.52      0.53      0.52      1536

[[223  80  40  29  10   2]
 [ 47 231  71  18  14   3]
 [ 21  49 174  41  20   2]
 [ 24   8  48 116  28   6]
 [ 23   7  17  36  55  16]
 [ 12  10  13   7  25  10]]
Epoch 8/8, Validation Loss: 1.1596, Validation Accuracy: 0.5267
Best model saved!
Finished Training
Best validation accuracy: 0.5267
